<a href="https://colab.research.google.com/github/AlexandreBourrieau/FICHIERS/blob/main/Multi_HRHN_SP500_PID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd

import random
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor

from keras import backend as K

# Initialisation TPU

In [ ]:
import os

use_tpu = True

if use_tpu:
    assert 'COLAB_TPU_ADDR' in os.environ, 'Missing TPU; did you request a TPU in Notebook Settings?'

if 'COLAB_TPU_ADDR' in os.environ:
  TPU_ADDRESS = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])
else:
  TPU_ADDRESS = ''

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=TPU_ADDRESS)
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

# Chargement et correction des données

Le dataset utilisé est SP500...

**1. Chargement des fichiers CSV**

In [ ]:
!rm *.csv
!curl --location --remote-header-name --remote-name "https://github.com/AlexandreBourrieau/FICHIERS/raw/main/Series_Temporelles/Multi/Data/SPX2010_2021.csv"
!curl --location --remote-header-name --remote-name "https://github.com/AlexandreBourrieau/FICHIERS/raw/main/Series_Temporelles/Multi/Data/VIX2010_2021.csv"
!curl --location --remote-header-name --remote-name "https://github.com/AlexandreBourrieau/FICHIERS/raw/main/Series_Temporelles/Multi/Data/US10Y2010_2021.csv"
!curl --location --remote-header-name --remote-name "https://github.com/AlexandreBourrieau/FICHIERS/raw/main/Series_Temporelles/Multi/Data/TYVIX2010_2021.csv"


**2. Analyse et correction des données S&P500**

In [ ]:
# Création de la série sous Pandas
df_SP500 = pd.read_csv("SPX2010_2021.csv",sep=";",names=["Date","SP500_O","SP500_H","SP500_L","SP500_C"],decimal=",")
df_SP500['Date'] = pd.to_datetime(df_SP500['Date'])
df_SP500 = df_SP500.set_index("Date")
df_SP500 = df_SP500.asfreq(freq="1D")
df_SP500

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=np.linspace(0,len(df_SP500),len(df_SP500)+1),y=df_SP500['SP500_C'], line=dict(color='blue', width=1),name="Index"))
fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

**2. Analyse et correction des données VIX**

In [ ]:
# Création de la série sous Pandas
df_VIX = pd.read_csv("VIX2010_2021.csv",sep=";",names=["Date","VIX_O","VIX_H","VIX_L","VIX_C"],decimal=",")
df_VIX['Date'] = pd.to_datetime(df_VIX['Date'])
df_VIX = df_VIX.set_index("Date")
df_VIX = df_VIX.asfreq(freq="1D")
df_VIX

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=np.linspace(0,len(df_VIX),len(df_VIX)+1),y=df_VIX['VIX_C'], line=dict(color='blue', width=1),name="Index"))
fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

**3. Analyse et correction des données US10Y**

In [ ]:
# Création de la série sous Pandas
df_US10Y= pd.read_csv("US10Y2010_2021.csv",sep=";",names=["Date","US10Y_O","US10Y_H","US10Y_L","US10Y_C"],decimal=",")
df_US10Y['Date'] = pd.to_datetime(df_US10Y['Date'])
df_US10Y = df_US10Y.set_index("Date")
df_US10Y = df_US10Y.asfreq(freq="1D")
df_US10Y

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=np.linspace(0,len(df_US10Y),len(df_US10Y)+1),y=df_US10Y['US10Y_C'], line=dict(color='blue', width=1),name="Index"))
fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

**4. Analyse et correction des données TYVIX**

In [ ]:
# Création de la série sous Pandas
df_TYVIX= pd.read_csv("TYVIX2010_2021.csv",sep=";",names=["Date","TYVIX_C"],decimal=",")
df_TYVIX['Date'] = pd.to_datetime(df_TYVIX['Date'])
df_TYVIX = df_TYVIX.set_index("Date")
df_TYVIX = df_TYVIX.asfreq(freq="1D")
df_TYVIX

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=np.linspace(0,len(df_TYVIX),len(df_TYVIX)+1),y=df_TYVIX['TYVIX_C'], line=dict(color='blue', width=1),name="Index"))
fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

**5. Fusion des données et correction des valeurs NaN**

In [ ]:
from functools import reduce

df_liste = [df_SP500,df_VIX,df_US10Y,df_TYVIX]

df_etude = reduce(lambda  left,right: pd.merge(left,right,on=['Date'],how='outer'), df_liste)
df_etude

In [ ]:
debut = "2011-01-10"
fin = "2020-04-24"

mask = (df_etude.index >= debut) & (df_etude.index <= fin)
df_etude = df_etude.loc[mask]
df_etude

In [ ]:
df_etude.isna().sum()

In [ ]:
df_etude = df_etude.interpolate(method="linear")
df_etude.isna().sum()

Déplace la cible à la fin :

In [ ]:
col = df_etude.pop('SP500_C')
df_etude.insert(len(df_etude.columns),"SP500_C",col)
df_etude

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=np.linspace(0,len(df_etude),len(df_etude)+1),y=df_etude['SP500_C'], line=dict(color='blue', width=1),name="Index"))
fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

# Séparation des données

In [ ]:
# Sépare les données en entrainement et tests
# 80% / 10% / 10%

temps_separation = int(len(df_etude.values) * 0.8)
temps_separation_PID = int(len(df_etude.values) * 0.9)
date_separation = df_etude.index[temps_separation]
date_separation_PID = df_etude.index[temps_separation_PID]

serie_entrainement_X = np.array(df_etude.values[:temps_separation],dtype=np.float32)
serie_test_X = np.array(df_etude.values[temps_separation:],dtype=np.float32)

serie_entrainement_PID_X = np.array(df_etude.values[temps_separation:temps_separation_PID],dtype=np.float32)
serie_test_PID_X = np.array(df_etude.values[temps_separation_PID:],dtype=np.float32)


print("Taille de l'entrainement : %d" %len(serie_entrainement_X))
print("Taille de la validation : %d" %len(serie_test_X))
print("Taille de l'entrainement PID : %d" %len(serie_entrainement_PID_X))
print("Taille de la validation PID: %d" %len(serie_test_PID_X))


**Normalisation des données :**

On normalise les données à l'aide de la fonction [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)

In [ ]:
from sklearn import preprocessing

# Constrution des séries
serie_entrainement_X_norm = []
serie_test_X_norm = []

serie_entrainement_PID_X_norm = []
serie_test_PID_X_norm = []


for i in range(0,len(df_etude.columns)):
  serie_entrainement_X_norm.append(serie_entrainement_X[:,i])
  serie_test_X_norm.append(serie_test_X[:,i])

  serie_entrainement_PID_X_norm.append(serie_entrainement_PID_X[:,i])
  serie_test_PID_X_norm.append(serie_test_PID_X[:,i])

serie_entrainement_X_norm = tf.convert_to_tensor(serie_entrainement_X_norm)
serie_entrainement_X_norm = tf.transpose(serie_entrainement_X_norm)
serie_test_X_norm = tf.convert_to_tensor(serie_test_X_norm)
serie_test_X_norm = tf.transpose(serie_test_X_norm)
serie_entrainement_PID_X_norm = tf.convert_to_tensor(serie_entrainement_PID_X_norm)
serie_entrainement_PID_X_norm = tf.transpose(serie_entrainement_PID_X_norm)
serie_test_PID_X_norm = tf.convert_to_tensor(serie_test_PID_X_norm)
serie_test_PID_X_norm = tf.transpose(serie_test_PID_X_norm)


# Initialisaton du MinMaxScaler
min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler.fit(serie_entrainement_X_norm)

# Normalisation des séries
serie_entrainement_X_norm = min_max_scaler.transform(serie_entrainement_X_norm)
serie_test_X_norm = min_max_scaler.transform(serie_test_X_norm)
serie_entrainement_PID_X_norm = min_max_scaler.transform(serie_entrainement_PID_X_norm)
serie_test_PID_X_norm = min_max_scaler.transform(serie_test_PID_X_norm)

In [ ]:
print(serie_entrainement_X_norm.shape)
print(serie_test_X_norm.shape)
print(serie_entrainement_PID_X_norm.shape)
print(serie_test_PID_X_norm.shape)


In [ ]:
# Affiche quelques séries
fig, ax = plt.subplots(constrained_layout=True, figsize=(15,5))

ax.plot(df_etude.index[:temps_separation].values,serie_entrainement_X_norm[:,0:1], label="X_Ent")
ax.plot(df_etude.index[temps_separation:temps_separation_PID].values,serie_entrainement_PID_X_norm[:,0:1], label="X_PID_Ent")
ax.plot(df_etude.index[temps_separation_PID:].values,serie_test_PID_X_norm[:,0:1], label="X_PID_Val")

ax.legend()
plt.show()

# Création des datasets

Les datasets sont créés de la manière suivante :

  <img src='https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Series_Temporelles/Multi/images/ConstructionDataset.png?raw=true' width=500/>

**1. Préparation des datasets**

In [ ]:
# Fonction permettant de créer un dataset à partir des données de la série temporelle
# X = {((X1_1,X1_2,...,X1_T),(X2_1,X2_2,...,X2_T),(X3_1,X3_2,...,X3_T)),
#       (Y1,Y2,...,YT)}
# Y = YT+1

def prepare_dataset_XY(seriesX, serieY, longueur_sequence, longueur_sortie, batch_size,shift):
  datasetX = tf.data.Dataset.from_tensor_slices(seriesX)
  datasetX = datasetX.window(longueur_sequence+longueur_sortie, shift=shift, drop_remainder=True)
  datasetX = datasetX.flat_map(lambda x: x.batch(longueur_sequence + longueur_sortie))
  datasetX = datasetX.map(lambda x: (x[0:longueur_sequence][:,:]))
  datasetX = datasetX.batch(batch_size,drop_remainder=True).prefetch(1)

  datasetY = tf.data.Dataset.from_tensor_slices(serieY)
  datasetY = datasetY.window(longueur_sequence+longueur_sortie, shift=shift, drop_remainder=True)
  datasetY = datasetY.flat_map(lambda x: x.batch(longueur_sequence + longueur_sortie))
  datasetY = datasetY.map(lambda x: (x[0:longueur_sequence][:,:]))
  datasetY = datasetY.batch(batch_size,drop_remainder=True).prefetch(1)

  datasetYPred = tf.data.Dataset.from_tensor_slices(serieY)
  datasetYPred = datasetYPred.window(longueur_sequence+longueur_sortie+1, shift=shift, drop_remainder=True)
  datasetYPred = datasetYPred.flat_map(lambda x: x.batch(longueur_sequence + longueur_sortie+1))
  datasetYPred = datasetYPred.map(lambda x: (x[0:-1][-1:,:]))
  datasetYPred = datasetYPred.batch(batch_size,drop_remainder=True).prefetch(1)


  dataset = tf.data.Dataset.zip((datasetX,datasetY))
  dataset = tf.data.Dataset.zip((dataset,datasetYPred))

  return dataset

In [ ]:
# Définition des caractéristiques du dataset que l'on souhaite créer
batch_size = 128
longueur_sequence = 10
longueur_sortie = 1
shift=1

# Création du dataset
dataset = prepare_dataset_XY(serie_entrainement_X_norm[:,0:-1],serie_entrainement_X_norm[:,-1:], longueur_sequence,longueur_sortie,batch_size,shift)
dataset_val = prepare_dataset_XY(serie_test_X_norm[:,0:-1],serie_test_X_norm[:,-1:],longueur_sequence,longueur_sortie,batch_size,shift)

dataset_PID = prepare_dataset_XY(serie_entrainement_PID_X_norm[:,0:-1],serie_entrainement_PID_X_norm[:,-1:], longueur_sequence,longueur_sortie,batch_size,shift)
dataset_PID_val = prepare_dataset_XY(serie_test_PID_X_norm[:,0:-1],serie_test_PID_X_norm[:,-1:],longueur_sequence,longueur_sortie,batch_size,shift)


In [ ]:
print(len(list(dataset.as_numpy_iterator())))
for element in dataset.take(1):
  print(element[0][0].shape)            # ((X1),(X2),...) = ((X1_1,X1_2,...,X1_T),(X2_1,X2_2,...,X2_T),...)
  print(element[0][1].shape)            # (Y1,Y2,...,YT)
  print(element[1].shape)               # YT+1

In [ ]:
print(len(list(dataset_val.as_numpy_iterator())))
for element in dataset_val.take(1):
  print(element[0][0].shape)            # ((X1),(X2),...) = ((X1_1,X1_2,...,X1_T),(X2_1,X2_2,...,X2_T),...)
  print(element[0][1].shape)            # Y1,Y2,...,YT
  print(element[1].shape)               # YT+1

In [ ]:
print(len(list(dataset_PID.as_numpy_iterator())))
for element in dataset_PID.take(1):
  print(element[0][0].shape)            # ((X1),(X2),...) = ((X1_1,X1_2,...,X1_T),(X2_1,X2_2,...,X2_T),...)
  print(element[0][1].shape)            # Y1,Y2,...,YT
  print(element[1].shape)               # YT+1

In [ ]:
print(len(list(dataset_PID_val.as_numpy_iterator())))
for element in dataset_PID_val.take(1):
  print(element[0][0].shape)            # ((X1),(X2),...) = ((X1_1,X1_2,...,X1_T),(X2_1,X2_2,...,X2_T),...)
  print(element[0][1].shape)            # Y1,Y2,...,YT
  print(element[1].shape)               # YT+1

**3. Préparation des X/Y**

In [ ]:
X1 = []
X2 = []

# Extrait les X,Y du dataset
x,y = tuple(zip(*dataset))              # x=43x((BS,10,3),(BS,9,1))
                                        # y=43x(BS,1,1)
for i in range(len(x)):
  X1.append(x[i][0])          
  X2.append(x[i][1])

X1 = tf.convert_to_tensor(X1)           # (43,BS,10,3)
X2 = tf.convert_to_tensor(X2)           # (43,BS,9,1)

X1 = np.asarray(X1,dtype=np.float32)    # (43,BS,10,3)
X2 = np.asarray(X2,dtype=np.float32)    # (43,BS,10,3)   

# Recombine les données
y = np.asarray(y,dtype=np.float32)      # 43x(BS,1,1) => (43xBS,1,1)
X1 = np.reshape(X1,(X1.shape[0]*X1.shape[1],X1.shape[2],X1.shape[3]))   # (43,BS,10,3) => (43xBS,10,3)
X2 = np.reshape(X2,(X2.shape[0]*X2.shape[1],X2.shape[2],X2.shape[3]))   # (43,BS,9,1) => (43*BS,9,1)

x_train = [X1,X2]
y_train = np.asarray(tf.reshape(y,shape=(y.shape[0]*y.shape[1],longueur_sortie,y.shape[3])))

# Affiche les formats
print(x_train[0].shape)
print(x_train[1].shape)
print(y_train.shape)


In [ ]:
X1 = []
X2 = []

# Extrait les X,Y du dataset
x,y = tuple(zip(*dataset_val))              # x=43x((BS,10,3),(BS,9,1))
                                        # y=43x(BS,1,1)
for i in range(len(x)):
  X1.append(x[i][0])          
  X2.append(x[i][1])

X1 = tf.convert_to_tensor(X1)           # (43,BS,10,3)
X2 = tf.convert_to_tensor(X2)           # (43,BS,9,1)

X1 = np.asarray(X1,dtype=np.float32)    # (43,BS,10,3)
X2 = np.asarray(X2,dtype=np.float32)    # (43,BS,10,3)   

# Recombine les données
y = np.asarray(y,dtype=np.float32)      # 43x(BS,1,1) => (43xBS,1,1)
X1 = np.reshape(X1,(X1.shape[0]*X1.shape[1],X1.shape[2],X1.shape[3]))   # (43,BS,10,3) => (43xBS,10,3)
X2 = np.reshape(X2,(X2.shape[0]*X2.shape[1],X2.shape[2],X2.shape[3]))   # (43,BS,9,1) => (43*BS,9,1)

x_val = [X1,X2]
y_val = np.asarray(tf.reshape(y,shape=(y.shape[0]*y.shape[1],longueur_sortie,y.shape[3])))

# Affiche les formats
print(x_val[0].shape)
print(x_val[1].shape)
print(y_val.shape)


In [ ]:
X1 = []
X2 = []

# Extrait les X,Y du dataset
x,y = tuple(zip(*dataset_PID))              # x=43x((BS,10,3),(BS,9,1))
                                        # y=43x(BS,1,1)
for i in range(len(x)):
  X1.append(x[i][0])          
  X2.append(x[i][1])

X1 = tf.convert_to_tensor(X1)           # (43,BS,10,3)
X2 = tf.convert_to_tensor(X2)           # (43,BS,9,1)

X1 = np.asarray(X1,dtype=np.float32)    # (43,BS,10,3)
X2 = np.asarray(X2,dtype=np.float32)    # (43,BS,10,3)   

# Recombine les données
y = np.asarray(y,dtype=np.float32)      # 43x(BS,1,1) => (43xBS,1,1)
X1 = np.reshape(X1,(X1.shape[0]*X1.shape[1],X1.shape[2],X1.shape[3]))   # (43,BS,10,3) => (43xBS,10,3)
X2 = np.reshape(X2,(X2.shape[0]*X2.shape[1],X2.shape[2],X2.shape[3]))   # (43,BS,9,1) => (43*BS,9,1)

x_train_PID = [X1,X2]
y_train_PID = np.asarray(tf.reshape(y,shape=(y.shape[0]*y.shape[1],longueur_sortie,y.shape[3])))

# Affiche les formats
print(x_train_PID[0].shape)
print(x_train_PID[1].shape)
print(y_train_PID.shape)


In [ ]:
X1 = []
X2 = []

# Extrait les X,Y du dataset
x,y = tuple(zip(*dataset_PID_val))              # x=43x((BS,10,3),(BS,9,1))
                                        # y=43x(BS,1,1)
for i in range(len(x)):
  X1.append(x[i][0])          
  X2.append(x[i][1])

X1 = tf.convert_to_tensor(X1)           # (43,BS,10,3)
X2 = tf.convert_to_tensor(X2)           # (43,BS,9,1)

X1 = np.asarray(X1,dtype=np.float32)    # (43,BS,10,3)
X2 = np.asarray(X2,dtype=np.float32)    # (43,BS,10,3)   

# Recombine les données
y = np.asarray(y,dtype=np.float32)      # 43x(BS,1,1) => (43xBS,1,1)
X1 = np.reshape(X1,(X1.shape[0]*X1.shape[1],X1.shape[2],X1.shape[3]))   # (43,BS,10,3) => (43xBS,10,3)
X2 = np.reshape(X2,(X2.shape[0]*X2.shape[1],X2.shape[2],X2.shape[3]))   # (43,BS,9,1) => (43*BS,9,1)

x_PID_val = [X1,X2]
y_PID_val = np.asarray(tf.reshape(y,shape=(y.shape[0]*y.shape[1],longueur_sortie,y.shape[3])))

# Affiche les formats
print(x_PID_val[0].shape)
print(x_PID_val[1].shape)
print(y_PID_val.shape)


# Création du modèle HRHN

Le modèle HRHN est décrit dans ce document de recherche : [Hierarchical Attention-Based Recurrent Highway Networks for Time Series Prediction](https://arxiv.org/pdf/1806.00685)

<img src='https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Series_Temporelles/Multi/images/Mod%C3%A8leHRHN1.png?raw=true' width=700>

**1. Création de l'encodeur**

L'encodeur a pour but de créer des représentations cachées des séries exogènes qui prennent en compte les relations spatiales entre ces séries ainsi que les relations temporelles.  
Les relations spatiales sont extraitent à l'aide d'un ensemble de réseaux de convolution qui produisent des représentations w1, w2... w(T-1).  
Ces représentations sont ensuites codées par un réseau RHN à 3 couches afin d'en extraire les relations temporelles. En sortie de ce réseau RHN, on extrait 3 tenseurs dont chacun contient les (T-1) états cachés de chaque couche du réseau RHN.

<img src='https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Series_Temporelles/Multi/images/HRHN_Encodeur_VueEnsemble.png?raw=true'>

***a. Création des CNN parallèlisés***

La structure d'un réseau de convolution est composée de trois couches CNN-1D + Max-pooling :

<img src='https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Series_Temporelles/Multi/images/HRHN_Encodeur_CNN1.png?raw=true'>

L'intégration de caque réseau dans Keras est parallélisée :

<img src='https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Series_Temporelles/Multi/images/HRHN_Encodeur_CNN2.png?raw=true'>

In [ ]:
# Arguments de la méthode __init__
#   dim_filtres_cnn   :   liste dimension des filtres ex: [3,3,3]
#   nbr_filtres_cnn   :   liste nbr de filtre sur chaque couche ex: [16,32,64]
#   dim_max_pooling   :   liste dimension max pooling après chaque couche ex: [3,3,3]

class Encodeur_CNN(tf.keras.layers.Layer):
  def __init__(self, dim_filtres_cnn, nbr_filtres_cnn, dim_max_pooling,dim_motif):
    self.dim_filtres_cnn = dim_filtres_cnn
    self.nbr_filtres_cnn = nbr_filtres_cnn
    self.dim_max_pooling = dim_max_pooling
    self.dim_motif = dim_motif
    super().__init__()                # Appel du __init__() de la classe Layer
  
  # Création de Tin réseaux de convolution + max_pooling en //
  ############################################################
  def build(self,input_shape):
    convs = []
    input_cnns = []

    # Création des Tin entrées des réseaux CNN
    for i in range(input_shape[1]):
        input_cnns.append(tf.keras.Input(shape=(input_shape[2],1)))       # input = Tin*(batch_size,#dim,1)

    # Création des Tin réseaux CNN
    for i in range(input_shape[1]):
      conv = tf.keras.layers.Conv1D(filters=self.nbr_filtres_cnn[0],      # conv : (batch_size,#dim,16)
                                    kernel_size=self.dim_filtres_cnn[0],
                                    activation='relu',
                                    padding='same',
                                    strides=1)(input_cnns[i])
      conv = tf.keras.layers.MaxPool1D(pool_size=self.dim_max_pooling[0],      # conv : (batch_size,#pooling1,16)
                                       padding='same')(conv)
      for n in range(1,len(self.dim_filtres_cnn)):
        conv = tf.keras.layers.Conv1D(filters=self.nbr_filtres_cnn[n],    # conv : (batch_size,#pooling_x,dim_filtres_cnn[n])
                                      kernel_size=self.dim_filtres_cnn[n],
                                      activation='relu',
                                      padding='same',
                                      strides=1)(conv)
        conv = tf.keras.layers.MaxPool1D(pool_size=self.dim_max_pooling[n],    # conv : (batch_size,#pooling_x,dim_filtres_cnn[n])
                                         padding='same')(conv)
      convs.append(conv)
    
    # Création de la sortie concaténée des Tin réseaux CNN
    out = tf.convert_to_tensor(convs)                                     # out : (Tin,batch_size,#pooling,64)
    out = tf.transpose(out,perm=[1,0,2,3])                                # out : (batch_size,Tin,#pooling,64)
    out = tf.keras.layers.Reshape(                                        # out : (batch_size,Tin,#pooling*64)
        target_shape=(out.shape[1],out.shape[2]*out.shape[3]))(out)

    if self.dim_motif == 0:
      out = tf.keras.layers.Dense(units=out.shape[2])(out)                  # out : (batch_size,Tin,dim_motif = #pooling*64) 
    else:
      out = tf.keras.layers.Dense(units=self.dim_motif)(out)                # out : (batch_size,Tin,dim_motif) 

    # Création du modèle global
    self.conv_model = tf.keras.Model(inputs=input_cnns,outputs=out)

    super().build(input_shape)        # Appel de la méthode build()
    
  # Entrées :
  #     input:  Entrée séries exogènes  : (batch_size,Tin,#dim)
  # Sorties :
  #     w:      Sorties des motifs CNN  : (batch_size,Tin,#dim_motif)
  #                                       (taille dernier filtre=64)
  def call(self, input):
    # Coupes temporelles sur les séries exogènes
    # au format : Tin*(batch_size,#dim,1)
    input_list = []
    for i in range(input.shape[1]):
      input_list.append(tf.transpose(input[:,i:i+1,:],perm=[0,2,1]))      # (batch_size,#dim,1)
    # Convolutions spatiales des séries exogènes
    w = self.conv_model(input_list)                                       # (batch_size,Tin,dim_motif)
    return w

***b. Création des cellules RHN***

<img src='https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Series_Temporelles/Multi/images/HRHN_Encodeur_RHN.png?raw=true'>

On crée une cellule RHN en reprenant le code précédent auquel :  
- On ajoute la possibilité de retourner tous les états cachés de chaque couche
- On ajoute la prise en compte de la dimension d'entrée correspondant à la dimension des motifs en sortie des réseaux CNN (dim_motif)

<img src='https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Series_Temporelles/Multi/images/Structure_RHN4.png?raw=true'>

In [ ]:
class Cellule_RHN(tf.keras.layers.Layer):
  def __init__(self, dim_RHN, nbr_couches, return_all_states = False, dim_input=1):
    self.dim_RHN = dim_RHN
    self.nbr_couches = nbr_couches
    self.dim_input = dim_input
    self.return_all_states = return_all_states
    super().__init__()                # Appel du __init__() de la classe Layer
  
  def build(self,input_shape):
    self.Wh = self.add_weight(shape=(input_shape[2],self.dim_RHN),initializer="normal",name="Wh")       # (#dim, #RHN)
    self.Wt = self.add_weight(shape=(input_shape[2],self.dim_RHN),initializer="normal",name="Wt")       # (#dim, #RHN)
    self.Wc = self.add_weight(shape=(input_shape[2],self.dim_RHN),initializer="normal",name="Wc")       # (#dim, #RHN)

    self.Rh = self.add_weight(shape=(self.nbr_couches,self.dim_RHN,self.dim_RHN),initializer="normal",name="Rh")      # (n_couches,#RHN, #RHN)
    self.Rt = self.add_weight(shape=(self.nbr_couches,self.dim_RHN,self.dim_RHN),initializer="normal",name="Rt")      # (n_couches,#RHN, #RHN)
    self.Rc = self.add_weight(shape=(self.nbr_couches,self.dim_RHN,self.dim_RHN),initializer="normal",name="Rc")      # (n_couches,#RHN, #RHN)

    self.bh = self.add_weight(shape=(self.nbr_couches,self.dim_RHN,1),initializer="normal",name="bh")        # (n_couches,#RHN, 1)
    self.bt = self.add_weight(shape=(self.nbr_couches,self.dim_RHN,1),initializer="normal",name="bt")        # (n_couches,#RHN, 1)
    self.bc = self.add_weight(shape=(self.nbr_couches,self.dim_RHN,1),initializer="normal",name="bc")        # (n_couches,#RHN, 1)

    super().build(input_shape)        # Appel de la méthode build()

    # Initialisation des masques de dropout
  def InitMasquesDropout(self,drop=0.0):
    self.Wh_ = tf.convert_to_tensor(np.random.binomial(n=1,p=1.0-drop,size=(self.dim_input,1)),dtype=tf.float32)                 # (#dim,1)
    self.Wt_ = tf.convert_to_tensor(np.random.binomial(n=1,p=1.0-drop,size=(self.dim_input,1)),dtype=tf.float32)                 # (#dim,1)
    self.Wc_ = tf.convert_to_tensor(np.random.binomial(n=1,p=1.0-drop,size=(self.dim_input,1)),dtype=tf.float32)                 # (#dim,1)
    self.Rh_ = tf.convert_to_tensor(np.random.binomial(n=1,p=1.0-drop,size=(self.nbr_couches,self.dim_RHN,1)),dtype=tf.float32)  # (n_couches,#RHN,1)
    self.Rt_ = tf.convert_to_tensor(np.random.binomial(n=1,p=1.0-drop,size=(self.nbr_couches,self.dim_RHN,1)),dtype=tf.float32)  # (n_couches,#RHN,1)
    self.Rc_ = tf.convert_to_tensor(np.random.binomial(n=1,p=1.0-drop,size=(self.nbr_couches,self.dim_RHN,1)),dtype=tf.float32)  # (n_couches,#RHN,1)

  # Entrées :
  #     input:          Entrées X[t]        : (batch_size,1,#dim)
  #     init_hidden:    Etat caché Init.    : (batch_size,#RHN)
  # Sorties :
  #     sL:             Etat caché de la dernière couche       : (batch_size,#RHN) 
  #           ou        Etats cachés de chaque couche SL[t]    : (batch_size,nbr_couches,#RHN)
  def call(self, input, init_hidden=None):
    # Construction d'un vecteur d'état nul si besoin
    if init_hidden == None:
      init_hidden = tf.matmul(tf.zeros(shape=(self.dim_RHN,input.shape[2])), # (#RHN,#dim)X(batch_size,#dim,1) = (batch_size,#RHN,1)
                              tf.transpose(input,perm=[0,2,1]))
      init_hidden = tf.squeeze(init_hidden,-1)                               # (batch_size,#RHN,1) => (batch_size,#RHN)
  
    liste_sl = []                                                            # Liste pour  enregistrer les états cachés de chaque couche
    # Calcul de hl, tl et cl
    for i in range(self.nbr_couches):
      if i==0:
        # Applique le masque aux poids
        Rh = tf.multiply(self.Rh_[0,:,:],self.Rh[0,:,:])                      # (#RHN,1)_x_(#RHN,#RHN) = (#RHN,#RHN)
        Rt = tf.multiply(self.Rt_[0,:,:],self.Rt[0,:,:])
        Rc = tf.multiply(self.Rc_[0,:,:],self.Rc[0,:,:])

        Wh = tf.multiply(self.Wh_,self.Wh)                                    # (#dim,1)_x_(#dim,#RHN) = (#dim,#RHN)
        Wt = tf.multiply(self.Wt_,self.Wt)
        Wc = tf.multiply(self.Wc_,self.Wc)
   
        # Calcul de hl
        hl = tf.matmul(Rh,tf.expand_dims(init_hidden,-1))                   # (#RHN,#RHN)X(batch_size,#RHN,1) = (batch_size,#RHN,1)
        hl = hl + self.bh[0,:,:]                                            # (batch_size,#RHN,1) + (#RHN,1) = (batch_size,#RHN,1)
        hl = hl + tf.matmul(tf.transpose(Wh),
                            tf.transpose(input,perm=[0,2,1]))               # (#RHN,#dim)X(batch_size,#dim,1) = (batch_size,#RHN,1)
        hl = tf.squeeze(hl,-1)                                              # (batch_size,#RHN)
        hl = K.tanh(hl)

        # Calcul de tl
        tl = tf.matmul(Rt,tf.expand_dims(init_hidden,-1))                   # (#RHN,#RHN)X(batch_size,#RHN,1) = (batch_size,#RHN,1)
        tl = tl + self.bt[0,:,:]                                            # (batch_size,#RHN,1) + (#RHN,1) = (batch_size,#RHN,1)
        tl = tl + tf.matmul(tf.transpose(Wt),
                            tf.transpose(input,perm=[0,2,1]))               # (#RHN,#dim)X(batch_size,#dim,1) = (batch_size,#RHN,1)
        tl = tf.squeeze(tl,-1)                                              # (batch_size,#RHN)
        tl = tf.keras.activations.sigmoid(tl)

        # Calcul de cl
        cl = tf.matmul(Rc,tf.expand_dims(init_hidden,-1))                   # (#RHN,#RHN)X(batch_size,#RHN,1) = (batch_size,#RHN,1)
        cl = cl + self.bc[0,:,:]                                            # (batch_size,#RHN,1) + (#RHN,1) = (batch_size,#RHN,1)
        cl = cl + tf.matmul(tf.transpose(Wc),
                            tf.transpose(input,perm=[0,2,1]))               # (#RHN,#dim)X(batch_size,#dim,1) = (batch_size,#RHN,1)
        cl = tf.squeeze(cl,-1)                                              # (batch_size,#RHN)
        cl = tf.keras.activations.sigmoid(cl)

      else:
        # Applique le masque aux poids
        Rh = tf.multiply(self.Rh_[i,:,:],self.Rh[i,:,:])
        Rt = tf.multiply(self.Rt_[i,:,:],self.Rt[i,:,:])
        Rc = tf.multiply(self.Rc_[i,:,:],self.Rc[i,:,:])

        # Calcul de hl
        hl = tf.matmul(Rh,tf.expand_dims(init_hidden,-1))                   # (#RHN,#RHN)X(batch_size,#RHN,1) = (batch_size,#RHN,1)
        hl = hl + self.bh[i,:,:]                                            # (batch_size,#RHN,1) + (#RHN,1) = (batch_size,#RHN,1)
        hl = tf.squeeze(hl,-1)                                              # (batch_size,#RHN)
        hl = K.tanh(hl)

        # Calcul de tl
        tl = tf.matmul(Rt,tf.expand_dims(init_hidden,-1))                   # (#RHN,#RHN)X(batch_size,#RHN,1) = (batch_size,#RHN,1)
        tl = tl + self.bt[i,:,:]                                            # (batch_size,#RHN,1) + (#RHN,1) = (batch_size,#RHN,1)
        tl = tf.squeeze(tl,-1)                                              # (batch_size,#RHN)
        tl = tf.keras.activations.sigmoid(tl)

        # Calcul de cl
        cl = tf.matmul(Rc,tf.expand_dims(init_hidden,-1))                   # (#RHN,#RHN)X(batch_size,#RHN,1) = (batch_size,#RHN,1)
        cl = cl + self.bc[i,:,:]                                            # (batch_size,#RHN,1) + (#RHN,1) = (batch_size,#RHN,1)
        cl = tf.squeeze(cl,-1)                                              # (batch_size,#RHN)
        cl = tf.keras.activations.sigmoid(cl)
      
      # Calcul de sl
      sl = tf.keras.layers.multiply([hl,tl])                                # (batch_size,#RHN)
      sl = sl + tf.keras.layers.multiply([init_hidden,cl])                  # (batch_size,#RHN)
      liste_sl.append(sl)       # Sauvegarde l'état caché de la couche courante
      init_hidden = sl
    if self.return_all_states == False:
      return sl
    else:
      liste_sl = tf.convert_to_tensor(liste_sl)                             # (nbr_couches,batch_size,#RHN)
      liste_sl = tf.transpose(liste_sl,perm=[1,0,2])                        # (batch_size,nbr_couches,#RHN)
      return liste_sl

***c. Création de l'encodeur : Convolutions + RHN***

<img src='https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Series_Temporelles/Multi/images/HRHN_Encodeur_VueEnsemble.png?raw=true'>

In [ ]:
# Arguments de la méthode __init__
#   dim_filtres_cnn   :   liste dimension des filtres ex: [3,3,3]
#   nbr_filtres_cnn   :   liste nbr de filtre sur chaque couche ex: [16,32,64]
#   dim_max_pooling   :   liste dimension max pooling après chaque couche ex: [3,3,3]
#   dim_motif         :   dimension du motif en sortie du CNN
#   dim_RHN           :   dimension du vecteur caché RHN
#   nbr_couches_RHN   :   nombre de couches du RHN
#   dropout           :   dropout variationnel pour le RHN ex: [0.1]

class Encodeur(tf.keras.layers.Layer):
  def __init__(self, dim_filtres_cnn, nbr_filtres_cnn, dim_max_pooling, dim_motif,dim_RHN,nbr_couches_RHN, dropout=0.0):
    self.dim_filtres_cnn = dim_filtres_cnn
    self.nbr_filtres_cnn = nbr_filtres_cnn
    self.dim_max_pooling = dim_max_pooling
    self.dim_motif = dim_motif
    self.dim_RHN = dim_RHN
    self.nbr_couches_RHN = nbr_couches_RHN
    self.dropout = dropout
    super().__init__()                # Appel du __init__() de la classe Layer
  
  def build(self,input_shape):
    self.encodeur_cnn = Encodeur_CNN(dim_filtres_cnn=self.dim_filtres_cnn,nbr_filtres_cnn=self.nbr_filtres_cnn,dim_max_pooling=self.dim_max_pooling,dim_motif=self.dim_motif)
    self.RHN = Cellule_RHN(dim_RHN=self.dim_RHN,nbr_couches=self.nbr_couches_RHN,return_all_states=True,dim_input=self.dim_motif)
    super().build(input_shape)        # Appel de la méthode build()
    
  # Entrées :
  #     input:          Entrées X         : (batch_size,Tin,#dim)
  # Sorties :
  #     hidden_states   Vecteurs cachés   : (batch_size,nbr_couches,Tin,#RHN)
  def call(self, input):
    # Convolutions spatiales des séries exogènes
    w = self.encodeur_cnn(input)      #  (batch_size,Tin,dim_motif)

    # Encodage des motifs CNN avec les cellules RHN
    sequence = []
    hidden = None

    # Initialisation des masques de dropout pour tous les pas de temps
    self.RHN.InitMasquesDropout(self.dropout)

    # Applique la cellule RHN à chaque pas de temps
    for i in range(input.shape[1]):
      hidden = self.RHN(w[:,i:i+1,:],hidden)          # Envoie (batch_size,1,dim_motif)
      sequence.append(hidden)                         # Sauve (batch_size,nbr_couches,#RHN)

      # Le premier état caché du prochain instant
      # est l'état caché de la dernière couche précédente
      hidden = hidden[:,self.nbr_couches_RHN-1,:]       # (batch_size,#RHN)

    # Traite le format des vecteurs cachés de l'encodeur
    sequence = tf.convert_to_tensor(sequence)               # (Tin,batch_size,nbr_couches,#RHN)
    hidden_states = tf.transpose(sequence,perm=[1,2,0,3])   # (batch_size,nbr_couches,Tin,#RHN)  

    return hidden_states

**Test de l'encodeur**

In [ ]:
x_train[0].shape

In [ ]:
x_train[0][0:5,:,:].shape

In [ ]:
s = Encodeur(dim_filtres_cnn=[16,32,64],nbr_filtres_cnn=[3,3,3],dim_max_pooling=[3,3,3],dim_motif=3,dim_RHN=128,nbr_couches_RHN=3,dropout=0.0)(x_train[0][0:5,:,:])
s.shape

In [ ]:
entrees_sequences = tf.keras.layers.Input(shape=(10,81))
sortie = Encodeur(dim_filtres_cnn=[16,32,64],nbr_filtres_cnn=[3,3,3],dim_max_pooling=[3,3,3],dim_motif=3,dim_RHN=128,nbr_couches_RHN=3,dropout=0.0)(entrees_sequences)
model = tf.keras.Model(entrees_sequences,sortie)

**2. Création du décodeur**

Le décodeur prend en entrée et à chaque pas de temps :  
- Le tenseur en sortie de l'encodeur RHN qui contient l'ensemble des vecteurs cachés des différentes couches : (batch_size,Nbr_couches,Tin,#RHN)
- L'état caché de la dernière couche du décodeur RHN précédent : (batch_size,#RHN)
- La valeur de la série cible à l'instant courant : (batch_size,1,1)

<img src='https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Series_Temporelles/Multi/images/HRHN_VueEnsembleDecodeur2.png?raw=true'>

**a. Création de la couche d'attention hiérarchique**

<img src='https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Series_Temporelles/Multi/images/HRHN_AttentionHierarchique.png?raw=true'>

On commence par créer la fonction permettant de calculer les scores. Cette fonction sera appelée avec la méthode TimeDistributed de Keras.

In [ ]:
class CalculScore(tf.keras.layers.Layer):
  def __init__(self):
    super().__init__()                # Appel du __init__() de la classe Layer
  
  def build(self,input_shape):
    self.T = self.add_weight(shape=(input_shape[1],input_shape[1]),initializer="normal",name="T")            # (#RHN, #RHN)
    self.U = self.add_weight(shape=(input_shape[1],input_shape[1]),initializer="normal",name="U")            # (#RHN, #RHN)
    self.b = self.add_weight(shape=(input_shape[1],1),initializer="normal",name="b")                         # (#RHN, 1)
    self.v = self.add_weight(shape=(input_shape[1],1),initializer="normal",name="v")                         # (#RHN, 1)
    super().build(input_shape)        # Appel de la méthode build()

  #     hid_state:  Etat initial RHN          : (batch_size,#RHN)
  def SetInitState(self,hid_state):
    self.hid_state = hid_state

  def compute_output_shape(self,input_shape):
    return(input_shape[0],1)

  # Entrées :
  #     input:      1 sortie encodeur RHN     : (batch_size,#RHN)
  # Sorties :
  #     score:      score                     : (batch_size,1,1)
  def call(self, input):
    score = tf.matmul(self.U,tf.expand_dims(input,-1))                      # (#RHN,#RHN)x(batch_size,#RHN,1) = (batch_size,#RHN,1)
    score = score + tf.matmul(self.T,tf.expand_dims(self.hid_state,-1))     # (batch_size,#RHN,1)
    score = score + self.b                                                  # (batch_size,#RHN,1)
    score = K.tanh(score)
    score = tf.matmul(tf.transpose(self.v),score)                           # (1,#RHN)x(batch_size,#RHN,1) = (batch_size,1,1)
    return tf.squeeze(score,-1)                                             # (batch_size,1)

On crée maintenant la couche d'attention hiérarchique :

In [ ]:
class AttentionHierarchique(tf.keras.layers.Layer):
  def __init__(self):
    super().__init__()                # Appel du __init__() de la classe Layer
  
  def build(self,input_shape):
    self.couche_score = CalculScore()
    super().build(input_shape)        # Appel de la méthode build()
    
  # Entrées :
  #     input:      Sorties d'une couche encodeur RHN       : (batch_size,Tin,#RHN)
  #     hid_state:  Etat initial RHN                        : (batch_size,#RHN)
  # Sorties :
  #     vc:         SousVecteur contexte                    : (batch_size,1,RHN)
  def call(self, input, hid_state):
    # Calcul des scores
    self.couche_score.SetInitState(hid_state)
    scores = tf.keras.layers.TimeDistributed(self.couche_score)(input)        # (batch_size,Tin,#RHN) : Timestep = Tin
                                                                              # (batch_size,#RHN) envoyé Tin fois
                                                                              # (batch_size,Tin,1) retourné
    scores = tf.keras.activations.softmax(scores,axis=1)                      # (batch_size,Tin,1)

    # Applique les scores aux sorties de la couche RHN
    poids = tf.multiply(input,scores)             # (batch_size,Tin,#RHN)_x_(batch_size,Tin,1) = (batch_size,Tin,#RHN)

    # Calcul le sous-vecteur contexte
    vc = K.sum(poids,axis=1)                      # (batch_size,#RHN)
    return tf.expand_dims(vc,1)                   # (batch_size,1,#RHN)

**b. Création du décodeur**

Dans le décodeur, on parallélise autant de couches d'attention que nécessaire afin de créer un modèle d'attention multi-entrées.

<img src='https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Series_Temporelles/Multi/images/ParaDecodeur.png?raw=true'>

In [ ]:
class Decodeur(tf.keras.layers.Layer):
  def __init__(self,dim_RHN,nbr_couches_RHN,dropout=0.0):
    self.dim_RHN = dim_RHN
    self.nbr_couches_RHN = nbr_couches_RHN
    self.dropout = dropout
    super().__init__()                # Appel du __init__() de la classe Layer
  
  def build(self,input_shape):
    attentions = []
    inputs_attention = []

    # Création des "nbr_couches" entrées des attentions
    # Chaque entrée est une liste : [input,init_state] = [((batch_size,Tin,#RHN)),((batch_size,#RHN))]
    for i in range(input_shape[1]):
      inputs_attention.append([tf.keras.Input(shape=(input_shape[2],input_shape[3])),          # input = "nbr_couches"*(batch_size,Tin,#RHN)
                                 tf.keras.Input(shape=(input_shape[3]))])                      # init_state = "nbr_couches"*(batch_size,#RHN)

    # Création des "nbr_couches" couches d'attentions hierarchiques
    for i in range(input_shape[1]):
      att = AttentionHierarchique()(inputs_attention[i][0],                 # inputs_attention[i][0] : (batch_size,Tin,#RHN)
                                    inputs_attention[i][1])                 # inputs_attention[i][1] : (batch_size,#RHN)
      attentions.append(att)

    # Création de la sortie concaténée des "nbr_couches" couches d'attentions
    out = tf.convert_to_tensor(attentions)                                # out : (nbr_couches,batch_size,1,#RHN)
    out = tf.transpose(out,perm=[1,0,2,3])                                # out : (batch_size,nbr_couches,1,#RHN)

    # Création du modèle global
    self.att_model = tf.keras.Model(inputs=inputs_attention,outputs=out)

    # Création des poids
    self.Wtilda = tf.keras.layers.Dense(units=1,activation=None,use_bias=None)
    self.Vtilda = tf.keras.layers.Dense(units=1,activation=None,use_bias=True)

    # Création du décodeur RHN
    self.dec_RHN = Cellule_RHN(dim_RHN=self.dim_RHN,nbr_couches=self.nbr_couches_RHN,return_all_states=False,dim_input=1)
   
    super().build(input_shape)        # Appel de la méthode build()
    
  # Entrées :
  #     input:      Sorties des couches de l'encodeur RHN   : (batch_size,nbr_couches,Tin,#RHN)
  #     hid_state:  Etat initial RHN                        : (batch_size,#RHN)
  #     Y:          Valeur de la série cible                : (batch_size,1)
  #     only_att    Si =True ne calcul que le vecteur ctx   : True/False
  # Sorties :
  #     d:          Vecteur contexte                        : (batch_size,nbr_couches*RHN)
  #     s:          Vecteur caché décodeur RHN              : (batch_size,#RHN)
  def call(self, input, hid_state, Y,only_att):
    # Initialisation de l'état caché à 0 si besoin
    # Construit le tenseur nul au format (batch_size,#RHN)
    if hid_state == None:
      coef = tf.expand_dims(input[:,0,0,0],-1)                        # (batch_size,1)
      coef = tf.expand_dims(coef,-1)                                  # (batch_size,1,1)
      hid_state = tf.matmul(coef,tf.zeros(shape=(1,input.shape[3])))  # (batch_size,1,1)X(1,#RHN) = (batch_size,1,#RHN)
      hid_state = tf.squeeze(hid_state,axis=1)                        # (batch_size,#RHN)

    # Construction de l'entrée du modèle
    # nbr_couches*[((batch_size,Tin,#RHN)),((batch_size,#RHN))]
    input_model = []
    for i in range(input.shape[1]):
      input_model.append([input[:,i,:,:],hid_state])    # [((batch_size,Tin,#RHN)),((batch_size,#RHN))]
    
    # Calcul des sous-vecteurs contextes
    # avec le modèle d'attention hiérarchique parallélisé
    d = self.att_model(input_model)                     # d : (batch_size,nbr_couches,1,#RHN)

    # Concaténation des sous-vecteurs contextes
    d = tf.squeeze(d,axis=2)                            # (batch_size,nbr_couches,#RHN)
    d = tf.keras.layers.Flatten()(d)                    # (batch_size,nbr_couches*RHN)

    if only_att == False :
      # Calcul de y_tilda
      ytilda = self.Wtilda(Y)                             # (batch_size,1)
      ytilda = ytilda + self.Vtilda(d)                    # (batch_size,1)

      # Initialisation des masques de dropout pour tous les pas de temps
      self.dec_RHN.InitMasquesDropout(self.dropout)

      # Décodage avec le réseau RHN
      s = self.dec_RHN(tf.expand_dims(ytilda,-1),hid_state)                  # (batch_size,#RHN)
      return d,s
    else:
      return d

**3. Création de la couche HRHN**

<img src='https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Series_Temporelles/Multi/images/Gene_HRHN.png?raw=true'>

In [ ]:
class Net_HRHN(tf.keras.layers.Layer):
  def __init__(self,encodeur,decodeur,longueur_sequence, dim_RHN, regul=0.0, drop = 0.0):
    self.encodeur = encodeur
    self.decodeur = decodeur
    self.longueur_sequence = longueur_sequence
    self.regul = regul
    self.drop = drop
    self.dim_RHN = dim_RHN
    super().__init__()                # Appel du __init__() de la classe Layer
  
  def build(self,input_shape):
    self.W = tf.keras.layers.Dense(units=1,activation=None,use_bias=None)
    self.V = tf.keras.layers.Dense(units=1,activation=None,use_bias=True)
    super().build(input_shape)        # Appel de la méthode build()

  # Entrées :
  #     input:          Entrées X           : (batch_size,Tin,#dim)
  #     output_seq:     Sortie séquence Y   : (batch_size,Tin,1)
  # Sorties :
  #     sortie:         Prédiction Y        : (batch_size,1,1)
  def call(self,input,output_seq):
    # Appel de l'encodeur
    # Récupère l'ensemble des états cachés de l'encodeur RHN
    H = self.encodeur(input)                # (batch_size,nbr_couches,Tin,#RHN)

    # Décodage
    hidden_state = None
    for t in range(input.shape[1]):
      vc, hidden_state = self.decodeur(H,hidden_state,output_seq[:,t:t+1,0],only_att = False)
    
    # Couche d'attention finale
    vc = self.decodeur(H,hidden_state,output_seq[:,0,0],only_att=True)

    # Génération de la prédiction
    sortie = self.W(hidden_state) + self.V(vc)        # (batch_size,1)
    return tf.expand_dims(sortie,-1)                  # (batch_size,1,1)

#Mise en place du modèle HRHN

In [ ]:
dim_RHN = 128
nbr_filtres_cnn = [16,32,64]
dim_filtres_cnn = [3,3,3]
dim_max_pooling = [3,3,3]
nbr_couches_RHN = 3
#dim_motif = 3
dim_motif = Encodeur_CNN(dim_filtres_cnn=dim_filtres_cnn,nbr_filtres_cnn=nbr_filtres_cnn,dim_max_pooling=dim_max_pooling,dim_motif=0)(x_train[0][0:1,:,:]).shape[2]
drop=0.0

def get_model_HRHN():
  entrees_sequences = tf.keras.layers.Input(shape=(longueur_sequence,x_train[0].shape[2]))
  sorties_sequence = tf.keras.layers.Input(shape=(longueur_sequence,1))

  encodeur = Encodeur(dim_filtres_cnn=dim_filtres_cnn,nbr_filtres_cnn=nbr_filtres_cnn,dim_max_pooling=dim_max_pooling,dim_motif=dim_motif,dim_RHN=dim_RHN,nbr_couches_RHN=nbr_couches_RHN,dropout=drop)
  decodeur = Decodeur(dim_RHN=dim_RHN,nbr_couches_RHN=nbr_couches_RHN,dropout=drop)

  sortie = Net_HRHN(encodeur,decodeur,longueur_sequence=longueur_sequence,dim_RHN=dim_RHN,drop=drop)(entrees_sequences,sorties_sequence)

  model = tf.keras.Model([entrees_sequences,sorties_sequence],sortie)
  return model

# Chargement du modèle pré-entrainé

**HRHN SP500 #1**  
  - longueur_sequence = 64      
  - dim_RHN = 128  
  - dim_filtres_cnn = [16,32,64]  
  - nbr_filtres_cnn = [2,2,2]  
  - dim_max_pooling = [2,2,2]  
  - nbr_couches_RHN = 3  
  - dim_motif = auto  
  - drop=0.00  
	=> mse : 4.2054e-06/0.0043   


In [ ]:
!rm *.hdf5
!curl --location --remote-header-name --remote-name "https://github.com/AlexandreBourrieau/FICHIERS/raw/main/Series_Temporelles/Multi/Models/Multi_HRHN_SP500_3.hdf5"
!curl --location --remote-header-name --remote-name "https://github.com/AlexandreBourrieau/FICHIERS/raw/main/Series_Temporelles/Multi/Models/Multi_HRHN_SP500_1.hdf5"


In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)
with strategy.scope():
  model_HRHN = get_model_HRHN()
  model_HRHN.compile(loss="mse")
  model_HRHN.load_weights("Multi_HRHN_SP500_1.hdf5")

# Prédictions single-step

In [ ]:
pred_ent = model_HRHN.predict([x_train[0],x_train[1]],verbose=1)
pred_val = model_HRHN.predict([x_val[0],x_val[1]],verbose=1)

In [ ]:
import plotly.graph_objects as go

decalage = 1

fig = go.Figure()

# Courbes originales
fig.add_trace(go.Scatter(x=df_etude.index,y=serie_entrainement_X_norm[:,-1],line=dict(color='blue', width=1)))
fig.add_trace(go.Scatter(x=df_etude.index[temps_separation:],y=serie_test_X_norm[:,-1],line=dict(color='red', width=1)))

#Affiche les prédictions sur l'entrainement
pred = []

max = len(pred_ent)
max = max
for i in range(0,max):
  pred.append(tf.squeeze(pred_ent[i,0:decalage,:],1))
pred = tf.convert_to_tensor(pred).numpy()
pred = np.reshape(pred,(pred.shape[0]*pred.shape[1]))

fig.add_trace(go.Scatter(x=df_etude.index[longueur_sequence:],y=pred, mode='lines', line=dict(color='green', width=1)))

#Affiche les prédictions sur les validations
pred = []
max = len(pred_val)
max = max
for i in range(0,max):
  pred.append(tf.squeeze(pred_val[i,0:decalage,:],1))

pred = tf.convert_to_tensor(pred).numpy()
pred = np.reshape(pred,(pred.shape[0]*pred.shape[1]))

fig.add_trace(go.Scatter(x=df_etude.index[temps_separation+longueur_sequence:],y=pred, mode='lines', line=dict(color='green', width=1)))

fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

**Zone d'entrainement du modèle HRHN**

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

decalage = 1
pred = []

max = len(pred_ent)
max = max
for i in range(0,max):
  pred.append(tf.squeeze(pred_ent[i,0:decalage,:],1))
pred = tf.convert_to_tensor(pred).numpy()
pred = np.reshape(pred,(pred.shape[0]*pred.shape[1]))

fig.add_trace(go.Scatter(x=df_etude.index[longueur_sequence:],y=serie_entrainement_X_norm[longueur_sequence:-(serie_entrainement_X_norm[longueur_sequence:,:].shape[0]-pred.shape[0]),-1],line=dict(color='blue', width=1)))
fig.add_trace(go.Scatter(x=df_etude.index[longueur_sequence:],y=pred,line=dict(color='green', width=1)))


fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

**Zone de validation du modèle HRHN**

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

decalage = 1
pred = []

max = len(pred_val)
max = max
for i in range(0,max):
  pred.append(tf.squeeze(pred_val[i,0:decalage,:],1))
pred = tf.convert_to_tensor(pred).numpy()
pred = np.reshape(pred,(pred.shape[0]*pred.shape[1]))

# Calcul les valeurs non prédites dans l'intervalle de validation
manque_pred = serie_test_X_norm[longueur_sequence:,:].shape[0]-pred.shape[0]

# Affiche les valeurs originales de l'intervalle de validation
fig.add_trace(go.Scatter(x=df_etude.index[temps_separation+longueur_sequence:-manque_pred:],y=serie_test_X_norm[longueur_sequence:-manque_pred,-1],line=dict(color='blue', width=1)))

# Affiche les prédictions sur l'intervalle d'entrainement PID
fig.add_trace(go.Scatter(x=df_etude.index[temps_separation+longueur_sequence:temps_separation_PID],y=pred[:temps_separation_PID-temps_separation-longueur_sequence],line=dict(color='green', width=1)))

# Affiche les prédictions sur l'intervalle de validation PID
fig.add_trace(go.Scatter(x=df_etude.index[temps_separation_PID:-manque_pred],y=pred[temps_separation_PID-temps_separation-longueur_sequence:],line=dict(color='red', width=1)))



fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

**Erreurs sur les zones PID**

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

decalage = 1
pred = []

max = len(pred_val)
max = max
for i in range(0,max):
  pred.append(tf.squeeze(pred_val[i,0:decalage,:],1))
pred = tf.convert_to_tensor(pred).numpy()
pred = np.reshape(pred,(pred.shape[0]*pred.shape[1]))

# Calcul les valeurs non prédites dans l'intervalle de validation
manque_pred = serie_test_X_norm[longueur_sequence:,:].shape[0]-pred.shape[0]

# Affiche les valeurs originales de l'intervalle d'entrainement PID
fig.add_trace(go.Scatter(x=df_etude.index[temps_separation+longueur_sequence:temps_separation_PID],y=serie_entrainement_PID_X_norm[longueur_sequence:,-1],line=dict(color='blue', width=1)))

# Affiche les prédictions sur l'intervalle d'entrainement PID
fig.add_trace(go.Scatter(x=df_etude.index[temps_separation+longueur_sequence:temps_separation_PID],y=pred[:temps_separation_PID-temps_separation-longueur_sequence],line=dict(color='green', width=1)))

fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

mse_PID_ent = tf.keras.losses.mse(serie_entrainement_PID_X_norm[longueur_sequence:,-1],pred[:temps_separation_PID-temps_separation-longueur_sequence])

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

decalage = 1
pred = []

max = len(pred_val)
max = max
for i in range(0,max):
  pred.append(tf.squeeze(pred_val[i,0:decalage,:],1))
pred = tf.convert_to_tensor(pred).numpy()
pred = np.reshape(pred,(pred.shape[0]*pred.shape[1]))

# Calcul les valeurs non prédites dans l'intervalle de validation
manque_pred = serie_test_X_norm[longueur_sequence:,:].shape[0]-pred.shape[0]

# Affiche les valeurs originales de l'intervalle de validation PID
fig.add_trace(go.Scatter(x=df_etude.index[temps_separation_PID:-manque_pred],y=serie_test_PID_X_norm[:-manque_pred,-1],line=dict(color='blue', width=1)))

# Affiche les prédictions sur l'intervalle de validation PID
fig.add_trace(go.Scatter(x=df_etude.index[temps_separation_PID:-manque_pred],y=pred[temps_separation_PID-temps_separation-longueur_sequence:],line=dict(color='red', width=1)))

fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

mse_PID_val = tf.keras.losses.mse(serie_test_PID_X_norm[:-manque_pred,-1],pred[temps_separation_PID-temps_separation-longueur_sequence:])

In [ ]:
print(mse_PID_ent)
print(mse_PID_val)

# Prédictions naïves

Mettons en place un modèle qui effectue des prédictions naïves sur le NASDAQ :

In [ ]:
class Net_Naif(tf.keras.layers.Layer):
  def __init__(self,longueur_sequence):
    self.longueur_sequence = longueur_sequence
    super().__init__()                # Appel du __init__() de la classe Layer
  
  def build(self,input_shape):
    super().build(input_shape)        # Appel de la méthode build()

  # Entrées :
  #     input:    Sortie séquence Y   : (batch_size,1,1)
  # Sorties :
  #     sortie:   Prédiction Y        : (batch_size,1,1)
  def call(self,input):
    return tf.expand_dims(input[:,-1:,:],-1)                  # (batch_size,1,1)

In [ ]:
def get_model_naif():
  sorties_sequence = tf.keras.layers.Input(shape=(longueur_sequence,1))
  sortie = Net_Naif(longueur_sequence=longueur_sequence)(sorties_sequence)
  model = tf.keras.Model(sorties_sequence,sortie)
  return model

In [ ]:
model_naif = get_model_naif()

In [ ]:
pred_ent_naif = model_naif.predict(x_train[1],verbose=1)
pred_val_naif = model_naif.predict(x_val[1],verbose=1)

In [ ]:
model_naif.compile(loss="mse")
model_naif.evaluate(x=x_train[1],y=y_train)
model_naif.evaluate(x=x_val[1],y=y_val)

In [ ]:
import plotly.graph_objects as go

decalage = 1

fig = go.Figure()

# Courbes originales
fig.add_trace(go.Scatter(x=df_etude.index,y=serie_entrainement_X_norm[:,-1],line=dict(color='blue', width=1)))
fig.add_trace(go.Scatter(x=df_etude.index[temps_separation:],y=serie_test_X_norm[:,-1],line=dict(color='red', width=1)))

#Affiche les prédictions sur l'entrainement
pred = []

max = len(pred_ent_naif)
max = max
for i in range(0,max):
  pred.append(tf.squeeze(pred_ent_naif[i,0:decalage,:],1))
pred = tf.convert_to_tensor(pred).numpy()
pred = np.reshape(pred,(pred.shape[0]*pred.shape[1]))

fig.add_trace(go.Scatter(x=df_etude.index[longueur_sequence:],y=pred, mode='lines', line=dict(color='green', width=1)))

#Affiche les prédictions sur les validations
pred = []
max = len(pred_val_naif)
max = max
for i in range(0,max):
  pred.append(tf.squeeze(pred_val_naif[i,0:decalage,:],1))

pred = tf.convert_to_tensor(pred).numpy()
pred = np.reshape(pred,(pred.shape[0]*pred.shape[1]))

fig.add_trace(go.Scatter(x=df_etude.index[temps_separation+longueur_sequence:],y=pred, mode='lines', line=dict(color='green', width=1)))

fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

**Erreurs en single step**

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

decalage = 1
pred = []

max = len(pred_val_naif)
max = max
for i in range(0,max):
  pred.append(tf.squeeze(pred_val_naif[i,0:decalage,:],1))
pred = tf.convert_to_tensor(pred).numpy()
pred = np.reshape(pred,(pred.shape[0]*pred.shape[1]))

# Calcul les valeurs non prédites dans l'intervalle de validation
manque_pred = serie_test_X_norm[longueur_sequence:,:].shape[0]-pred.shape[0]

# Affiche les valeurs originales de l'intervalle d'entrainement PID
fig.add_trace(go.Scatter(x=df_etude.index[temps_separation+longueur_sequence:temps_separation_PID],y=serie_entrainement_PID_X_norm[longueur_sequence:,-1],line=dict(color='blue', width=1)))

# Affiche les prédictions sur l'intervalle d'entrainement PID
fig.add_trace(go.Scatter(x=df_etude.index[temps_separation+longueur_sequence:temps_separation_PID],y=pred[:temps_separation_PID-temps_separation-longueur_sequence],line=dict(color='green', width=1)))

fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

mse_PID_ent_naif = tf.keras.losses.mse(serie_entrainement_PID_X_norm[longueur_sequence:,-1],pred[:temps_separation_PID-temps_separation-longueur_sequence])

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

decalage = 1
pred = []

max = len(pred_val_naif)
max = max
for i in range(0,max):
  pred.append(tf.squeeze(pred_val_naif[i,0:decalage,:],1))
pred = tf.convert_to_tensor(pred).numpy()
pred = np.reshape(pred,(pred.shape[0]*pred.shape[1]))

# Calcul les valeurs non prédites dans l'intervalle de validation
manque_pred = serie_test_X_norm[longueur_sequence:,:].shape[0]-pred.shape[0]

# Affiche les valeurs originales de l'intervalle de validation PID
fig.add_trace(go.Scatter(x=df_etude.index[temps_separation_PID:-manque_pred],y=serie_test_PID_X_norm[:-manque_pred,-1],line=dict(color='blue', width=1)))

# Affiche les prédictions sur l'intervalle de validation PID
fig.add_trace(go.Scatter(x=df_etude.index[temps_separation_PID:-manque_pred],y=pred[temps_separation_PID-temps_separation-longueur_sequence:],line=dict(color='red', width=1)))

fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

mse_PID_val_naif = tf.keras.losses.mse(serie_test_PID_X_norm[:-manque_pred,-1],pred[temps_separation_PID-temps_separation-longueur_sequence:])

In [ ]:
print(mse_PID_ent_naif)
print(mse_PID_val_naif)

# Création du modèle HRHN + CORRECTEUR PID

**1. Couche du correcteur**

In [ ]:
class Correcteur_PID(tf.keras.layers.Layer):
  def __init__(self):
    super().__init__()                # Appel du __init__() de la classe Layer
  
  def build(self,input_shape):
    self.Couche_Dense = tf.keras.layers.Dense(units=1)
    super().build(input_shape)        # Appel de la méthode build()
  
  # y_reel :    Valeur cible réelle à (t-1)   :   (batch_size,1,1)
  # y_pred :    Prédiction à (t-1)            :   (batch_size,1,1)
  def call(self, y_reel,y_pred):
    return self.Couche_Dense(y_reel-y_pred)

**2. Couche du prédicteur**

In [ ]:
class Predicteur(tf.keras.layers.Layer):
  def __init__(self,predicteur):
    self.predicteur = predicteur
    super().__init__()                # Appel du __init__() de la classe Layer
  
  def build(self,input_shape):
    super().build(input_shape)        # Appel de la méthode build()
  
  # input:    input[0] :  Séries exogènes     : (batch_size,Tin,#dim)
  #           input[1] :  Cible               : (batch_size,Tin,1)
  def call(self, input):
    return self.predicteur(input)

**2. Création du modèle**

In [ ]:
class CustomModel(keras.Model):
    def __init__(self,predicteur,correcteur):
      super(CustomModel, self).__init__()
      self.predicteur = predicteur
      self.Correcteur = correcteur
      self.y_pred_ent_1 = tf.Variable(shape=(1,1,1),trainable=False,initial_value=tf.zeros(shape=(1,1,1)))          # (1, 1)

    # inputs:   inputs[0] : Séries exogènes     : (Tin,#dim)
    #           inputs[1] : Cible               : (Tin,1)
    def call(self, inputs, training=True):
      y_pred = self.predicteur([inputs[0],inputs[1]])
      y_1 = inputs[1][:,-1:,:]                                # Vraie valeur précédente : (batch_size,1,1)
      y_pred_ent_1 = self.y_pred_ent_1
      offset = self.Correcteur(y_1,y_pred_ent_1)              # Calcul de l'offset : (batch_size,1,1)
      y_pred = y_pred + offset                                # Ajoute l'offset à la prédiction
      self.y_pred_ent_1.assign(y_pred)                        # Enregistre la prédiction
      return y_pred

assert tf.distribute.get_replica_context() is not None  # default

In [ ]:
model_HRHN = get_model_HRHN()
model_HRHN.compile(loss="mse")
model_HRHN.load_weights("Multi_HRHN_SP500_1.hdf5")
model_HRHN.trainable = False
model_HRHN.summary()

In [ ]:
with strategy.scope():
  predicteur = Predicteur(model_HRHN)
  correcteur = Correcteur_PID()
  model = CustomModel(predicteur,correcteur)

  periodes = 10
  optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
  loss_fn = keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.SUM)
  loss_metric = tf.keras.metrics.Mean()


  def train_step():
    with tf.GradientTape() as tape:
      pred = model([x_train_PID[0][i:i+1,:,:],x_train_PID[1][i:i+1,:,:]])
      loss = loss_fn(pred, y_train_PID[i:i+1,:,:])
    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return loss

  @tf.function
  def distributed_train_step():
#    assert tf.distribute.get_replica_context() is None
    per_replica_losses = strategy.run(train_step)
    return per_replica_losses


In [ ]:
for periode in range(periodes):
  print(periode)
  for i in range(x_train_PID[0].shape[0]):
    loss = distributed_train_step()


In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)
strategy.run(train)

# Entrainement avec TPU

In [ ]:
x_train_PID[1][:,-1:,:].shape

In [ ]:
from google.colab import files

max_periodes = 100

strategy = tf.distribute.TPUStrategy(resolver)
with strategy.scope():
  # Création du modèle
  model_HRHN = get_model_HRHN()
  model_HRHN.compile(loss="mse")
  model_HRHN.load_weights("Multi_HRHN_SP500_1.hdf5")
  model_HRHN.trainable = False

  cor_erreur = tf.keras.Sequential(
      [
      tf.keras.layers.InputLayer(input_shape=(1,1)),
      tf.keras.layers.Dense(units=1,trainable=True),
      ],
      name="cor_erreur"
  )
  entree = tf.keras.layers.Input(input_shape=(longueur_sequence,))


  model = CustomModel(model_HRHN,cor_erreur,batch_size)

  # Définition des paramètres liés à l'évolution du taux d'apprentissage
  lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
      initial_learning_rate=0.001,
      decay_steps=20,
      decay_rate=0.01)

  optimiseur=tf.keras.optimizers.Adam(learning_rate=lr_schedule)

  # Utilisation de la méthode ModelCheckPoint
  CheckPoint = tf.keras.callbacks.ModelCheckpoint("poids_train.hdf5", monitor='loss', verbose=1, save_best_only=True, save_weights_only = True, mode='auto', save_freq='epoch')

  # Compile le modèle
  model.compile(loss="mse", optimizer=optimiseur)

  # Entraine le modèle
  historique = model.fit(x=[x_train_PID[0],x_train_PID[1]],y=y_train_PID,validation_data=([x_PID_val[0],x_PID_val[1]],y_PID_val), epochs=max_periodes,verbose=1, callbacks=[CheckPoint,tf.keras.callbacks.EarlyStopping(monitor='loss', patience=100)])

In [ ]:
model.load_weights("poids_train.hdf5")

In [ ]:
erreur_entrainement = historique.history["loss"]
erreur_validation = historique.history["val_loss"]

# Affiche l'erreur en fonction de la période
plt.figure(figsize=(10, 6))
plt.plot(np.arange(0,len(erreur_entrainement)),erreur_entrainement, label="Erreurs sur les entrainements")
plt.plot(np.arange(0,len(erreur_entrainement)),erreur_validation, label ="Erreurs sur les validations")
plt.legend()

plt.title("Evolution de l'erreur en fonction de la période")

In [ ]:
erreur_entrainement = historique.history["loss"]
erreur_validation = historique.history["val_loss"]

# Affiche l'erreur en fonction de la période
plt.figure(figsize=(10, 6))
plt.plot(np.arange(0,len(erreur_entrainement[40:100])),erreur_entrainement[40:100], label="Erreurs sur les entrainements")
plt.plot(np.arange(0,len(erreur_entrainement[40:100])),erreur_validation[40:100], label ="Erreurs sur les validations")
plt.legend()

plt.title("Evolution de l'erreur en fonction de la période")

# Prédictions single-step

In [ ]:
pred_ent_PID = model.predict([x_train[0],x_train[1]],verbose=1)
pred_val_PID = model.predict([x_val[0],x_val[1]],verbose=1)

In [ ]:
import plotly.graph_objects as go

decalage = 1

fig = go.Figure()

# Courbes originales
fig.add_trace(go.Scatter(x=df_etude.index,y=serie_entrainement_X_norm[:,-1],line=dict(color='blue', width=1)))
fig.add_trace(go.Scatter(x=df_etude.index[temps_separation:],y=serie_test_X_norm[:,-1],line=dict(color='red', width=1)))

#Affiche les prédictions sur l'entrainement
pred = []

max = len(pred_ent_PID)
max = max
for i in range(0,max):
  pred.append(tf.squeeze(pred_ent_PID[i,0:decalage,:],1))
pred = tf.convert_to_tensor(pred).numpy()
pred = np.reshape(pred,(pred.shape[0]*pred.shape[1]))

fig.add_trace(go.Scatter(x=df_etude.index[longueur_sequence:],y=pred, mode='lines', line=dict(color='green', width=1)))

#Affiche les prédictions sur les validations
pred = []
max = len(pred_val_PID)
max = max
for i in range(0,max):
  pred.append(tf.squeeze(pred_val_PID[i,0:decalage,:],1))

pred = tf.convert_to_tensor(pred).numpy()
pred = np.reshape(pred,(pred.shape[0]*pred.shape[1]))

fig.add_trace(go.Scatter(x=df_etude.index[temps_separation+longueur_sequence:],y=pred, mode='lines', line=dict(color='green', width=1)))

fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

**Zone d'entrainement du modèle HRHN**

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

decalage = 1
pred = []

max = len(pred_ent_PID)
max = max
for i in range(0,max):
  pred.append(tf.squeeze(pred_ent_PID[i,0:decalage,:],1))
pred = tf.convert_to_tensor(pred).numpy()
pred = np.reshape(pred,(pred.shape[0]*pred.shape[1]))

fig.add_trace(go.Scatter(x=df_etude.index[longueur_sequence:],y=serie_entrainement_X_norm[longueur_sequence:-(serie_entrainement_X_norm[longueur_sequence:,:].shape[0]-pred.shape[0]),-1],line=dict(color='blue', width=1)))
fig.add_trace(go.Scatter(x=df_etude.index[longueur_sequence:],y=pred,line=dict(color='green', width=1)))


fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

**Zone de validation du modèle HRHN**

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

decalage = 1
pred = []

max = len(pred_val_PID)
max = max
for i in range(0,max):
  pred.append(tf.squeeze(pred_val_PID[i,0:decalage,:],1))
pred = tf.convert_to_tensor(pred).numpy()
pred = np.reshape(pred,(pred.shape[0]*pred.shape[1]))

# Calcul les valeurs non prédites dans l'intervalle de validation
manque_pred = serie_test_X_norm[longueur_sequence:,:].shape[0]-pred.shape[0]

# Affiche les valeurs originales de l'intervalle de validation
fig.add_trace(go.Scatter(x=df_etude.index[temps_separation+longueur_sequence:-manque_pred:],y=serie_test_X_norm[longueur_sequence:-manque_pred,-1],line=dict(color='blue', width=1)))

# Affiche les prédictions sur l'intervalle d'entrainement PID
fig.add_trace(go.Scatter(x=df_etude.index[temps_separation+longueur_sequence:temps_separation_PID],y=pred[:temps_separation_PID-temps_separation-longueur_sequence],line=dict(color='green', width=1)))

# Affiche les prédictions sur l'intervalle de validation PID
fig.add_trace(go.Scatter(x=df_etude.index[temps_separation_PID:-manque_pred],y=pred[temps_separation_PID-temps_separation-longueur_sequence:],line=dict(color='red', width=1)))



fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

**Erreurs sur les zones PID**

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

decalage = 1
pred = []

max = len(pred_val_PID)
max = max
for i in range(0,max):
  pred.append(tf.squeeze(pred_val_PID[i,0:decalage,:],1))
pred = tf.convert_to_tensor(pred).numpy()
pred = np.reshape(pred,(pred.shape[0]*pred.shape[1]))

# Calcul les valeurs non prédites dans l'intervalle de validation
manque_pred = serie_test_X_norm[longueur_sequence:,:].shape[0]-pred.shape[0]

# Affiche les valeurs originales de l'intervalle d'entrainement PID
fig.add_trace(go.Scatter(x=df_etude.index[temps_separation+longueur_sequence:temps_separation_PID],y=serie_entrainement_PID_X_norm[longueur_sequence:,-1],line=dict(color='blue', width=1)))

# Affiche les prédictions sur l'intervalle d'entrainement PID
fig.add_trace(go.Scatter(x=df_etude.index[temps_separation+longueur_sequence:temps_separation_PID],y=pred[:temps_separation_PID-temps_separation-longueur_sequence],line=dict(color='green', width=1)))

fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

mse_PID_ent = tf.keras.losses.mse(serie_entrainement_PID_X_norm[longueur_sequence:,-1],pred[:temps_separation_PID-temps_separation-longueur_sequence])

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

decalage = 1
pred = []

max = len(pred_val_PID)
max = max
for i in range(0,max):
  pred.append(tf.squeeze(pred_val_PID[i,0:decalage,:],1))
pred = tf.convert_to_tensor(pred).numpy()
pred = np.reshape(pred,(pred.shape[0]*pred.shape[1]))

# Calcul les valeurs non prédites dans l'intervalle de validation
manque_pred = serie_test_X_norm[longueur_sequence:,:].shape[0]-pred.shape[0]

# Affiche les valeurs originales de l'intervalle de validation PID
fig.add_trace(go.Scatter(x=df_etude.index[temps_separation_PID:-manque_pred],y=serie_test_PID_X_norm[:-manque_pred,-1],line=dict(color='blue', width=1)))

# Affiche les prédictions sur l'intervalle de validation PID
fig.add_trace(go.Scatter(x=df_etude.index[temps_separation_PID:-manque_pred],y=pred[temps_separation_PID-temps_separation-longueur_sequence:],line=dict(color='red', width=1)))

fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

mse_PID_val = tf.keras.losses.mse(serie_test_PID_X_norm[:-manque_pred,-1],pred[temps_separation_PID-temps_separation-longueur_sequence:])

In [ ]:
print(mse_PID_ent)
print(mse_PID_val)